In [1]:
# unset PYTHONPATH first
from ko_text import *
from ko_crawler import *

C:\Users\Dong\AppData\Local\Continuum\anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)
C:\Users\Dong\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
nlp = NLP()

In [4]:
train_df = pd.read_csv('Data/Train_final.csv', encoding = 'cp949')
test_df = pd.read_csv('Data/Test_final.csv', encoding = 'cp949')

# 용량을 줄이기 위해 '단어 단어' 꼴로 묶어둔 token을 ['단어', '단어'] 꼴로 풀기
train_df['Token'] = [token.split() for token in train_df['Token']]
test_df['Token'] = [token.split() for token in test_df['Token']]

In [5]:
print(train_df.shape)

(87899, 2)


In [6]:
Counter(train_df['Section'])

Counter({'IT과학': 9996,
         '경제': 9660,
         '국제': 9844,
         '기업': 9965,
         '문화': 9846,
         '부동산': 9986,
         '사회': 9880,
         '정치': 9295,
         '증권': 9427})

In [7]:
train_df.head()

,Token,Section
0,"[NH, 농협, 금융, 지주, 김광수, 회장, 22일, 서대문, 본사, 에서, 농협...",경제
1,"[바이오, 직스, 올해, 개별, 기준, 3분, 영업, 이익, 105억원, 으로, 작...",경제
2,"[코스피, 붕괴하면서, 달러, 환율, 으로, 뛰었다, 23일, 서울, 외환시장, 에...",경제
3,"[정의당, 국회, 롯데, 갑질, 피해자, 간담, 개최, 김상조, 정거, 위원장, 2...",경제
4,"[코스피, 23일, 무역, 전쟁, 내외, 악재, 힘없이, 2110, 내줬다, 코스피...",경제


In [8]:
# 한 label마다 학습할 단어의 수
train_size_for_each_label = 1000
test_size_for_each_label = 300

n_class = len(train_df['Section'].unique())


# Doc2Vec으로 vector를 추정하기 위한 split 과정
train_batch_size = n_class * train_size_for_each_label
test_batch_size = n_class * test_size_for_each_label

train_token_ls_split, train_tag_ls_split = nlp.extract_a_equally_splited_batch(train_df['Token'], train_df['Section'], train_batch_size)
test_token_ls_split, test_tag_ls_split =  nlp.extract_a_equally_splited_batch(test_df['Token'],test_df['Section'], test_batch_size)

<br>


<br>


In [9]:
' '.join(train_token_ls_split[0])

'NH 농협 금융 지주 김광수 회장 22일 서대문 본사 에서 농협 금융 자회사 대표 3분 종합 경영 성과 분석 회의 했다고 23일 밝혔다 회의 에서는 NH 농협카드 NH 농협 캐피탈 체질개선 질적 성장 방안 주로 논의 됐다 회장 농협카드 조직 효율 성과 인적 전문성 높이 독립 성과 평가 통해 책임 경영 체제 확립 라고 주문 했다 상품 고객 수익 분석 고도화 일반 카드 수준 경영 인프라 축하 라고 당부 했다 회장 NH 농협 캐피탈 관련 해서는 산업 금융 건설 경기 악화 염두 두고 자산 건전성 개선 노력 어가 기업 금융 취급 부터 위험관리 철저히 해야 한다고 강조 했다 회장 연말 인사 에서는 업무 경력 직무 전문성 우선 고려 하겠다고 했다'

In [10]:
train_tag_ls_split[0]

'경제'

# Doc2Vec Parameter 튜닝

## **set ALPHA as default**

In [ ]:
%%time

result_dict = {'corpus_count' : [],
               'min_count' : [],
               'vector_size' : [],
               'window' : [],
               'n_epochs' : [],
               'accuracy' : [],
              }

testing_section_ls = ['경제','기업','사회','국제','부동산','증권','정치','IT과학','문화']

# 하이퍼 파라미터 튜닝 작업 수행

#for size in [1000,10000, 30000]:
for size in [len(train_df)]:
    x_split, y_split = nlp.extract_a_equally_splited_batch(train_df['Token'].tolist(), train_df['Section'].tolist(), size)

    for min_count in [10, 30, 50]:
        for vector_size in [100, 300]:
            for window in [5, 10]:
                for n_epochs in [10,30]:

                    # Doc2Vec 모델 생성
                    nlp.make_Doc2Vec_model(min_count = min_count,
                                           sample = 1e-05,
                                           vector_size = vector_size,
                                           window = window,
                                           dm_mean = 0,
                                           dm_concat = 0)

                    nlp.build_and_train_Doc2Vec_model(x_split,
                                                      y_split,
                                                      n_epochs = n_epochs)


                    model_name = 'Doc2Vec_dm=%s&cc=%s&vs=%s&win=%s&neg=%s&min=%s&sample=%s&epochs=%s'%(\
                                                                                                         nlp.Doc2Vec_model.dm,
                                                                                                           nlp.Doc2Vec_model.corpus_count,
                                                                                                           nlp.Doc2Vec_model.vector_size,
                                                                                                           nlp.Doc2Vec_model.window,
                                                                                                           nlp.Doc2Vec_model.negative,
                                                                                                           nlp.Doc2Vec_model.min_count,
                                                                                                           nlp.Doc2Vec_model.sample,
                                                                                                           nlp.Doc2Vec_model.epochs)
                    # Doc2Vec 모델 저장
                    #nlp.Doc2Vec_model.save('Doc2Vec_model/'+model_name)


                    '''
                    
                    X =nlp.infer_vectors_with_Doc2Vec(train_token_for_display[0])

                    from sklearn.decomposition import PCA

                    pca = PCA(n_components=2)
                    X_pca = pca.fit_transform(X)
                    scatter_df = pd.DataFrame(X_pca,
                                              index = train_tag_for_display[0],
                                              columns = ['x','y'])

                    plt.figure(figsize = (15, 15))

                    for section in testing_section_ls:
                        temp_df = scatter_df[scatter_df.index == section]
                        plt.scatter(temp_df['x'].values, temp_df['y'].values, label = section, c = np.random.rand(3,))

                    plt.legend(loc = 'best')
                    plt.savefig('Doc2Vec_model/images/'+model_name)

                    '''
                    # 각 레이블별 1000개씩 학습, 

                    # LR
                    X_train = nlp.infer_vectors_with_Doc2Vec(train_token_ls_split)
                    y_train = train_tag_ls_split

                    X_test = nlp.infer_vectors_with_Doc2Vec(test_token_ls_split)
                    y_test = test_tag_ls_split


                    clf = LogisticRegression(solver = 'sag',
                                             multi_class = 'multinomial')


                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_test)

                    result_dict['corpus_count'].append(nlp.Doc2Vec_model.corpus_count)
                    result_dict['min_count'].append(nlp.Doc2Vec_model.min_count)
                    result_dict['vector_size'].append(nlp.Doc2Vec_model.vector_size)
                    result_dict['window'].append(nlp.Doc2Vec_model.window)
                    result_dict['n_epochs'].append(nlp.Doc2Vec_model.epochs)
                    result_dict['accuracy'].append(accuracy_score(y_pred, y_test))
                    
                    print(accuracy_score(y_pred, y_test))
                    
                    
    pd.DataFrame(result_dict).to_csv('Parameter_tuning_result.csv', index = False)

2018-11-13 21:44:49,903 : INFO : collecting all words and their counts
2018-11-13 21:44:49,904 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-13 21:44:50,211 : INFO : PROGRESS: at example #10000, processed 2042917 words (6654563/s), 70678 word types, 3 tags
2018-11-13 21:44:50,576 : INFO : PROGRESS: at example #20000, processed 4298916 words (6199131/s), 105924 word types, 5 tags
2018-11-13 21:44:50,907 : INFO : PROGRESS: at example #30000, processed 6453383 words (6524872/s), 126722 word types, 7 tags
2018-11-13 21:44:51,292 : INFO : PROGRESS: at example #40000, processed 8869096 words (6282073/s), 153964 word types, 9 tags
2018-11-13 21:44:51,684 : INFO : PROGRESS: at example #50000, processed 11118225 words (5738983/s), 183614 word types, 9 tags
2018-11-13 21:44:52,031 : INFO : PROGRESS: at example #60000, processed 13192619 words (5987882/s), 197009 word types, 9 tags
2018-11-13 21:44:52,408 : INFO : PROGRESS: at example #70000, processed 15

0.6888888888888889


2018-11-13 21:47:36,452 : INFO : collecting all words and their counts
2018-11-13 21:47:36,453 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-13 21:47:36,758 : INFO : PROGRESS: at example #10000, processed 2042917 words (6717176/s), 70678 word types, 3 tags
2018-11-13 21:47:37,119 : INFO : PROGRESS: at example #20000, processed 4298916 words (6271868/s), 105924 word types, 5 tags
2018-11-13 21:47:37,446 : INFO : PROGRESS: at example #30000, processed 6453383 words (6596806/s), 126722 word types, 7 tags
2018-11-13 21:47:37,827 : INFO : PROGRESS: at example #40000, processed 8869096 words (6344017/s), 153964 word types, 9 tags
2018-11-13 21:47:38,213 : INFO : PROGRESS: at example #50000, processed 11118225 words (5833061/s), 183614 word types, 9 tags
2018-11-13 21:47:38,549 : INFO : PROGRESS: at example #60000, processed 13192619 words (6194533/s), 197009 word types, 9 tags
2018-11-13 21:47:38,923 : INFO : PROGRESS: at example #70000, processed 15

0.677037037037037


2018-11-13 21:54:07,463 : INFO : collecting all words and their counts
2018-11-13 21:54:07,463 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-13 21:54:07,770 : INFO : PROGRESS: at example #10000, processed 2042917 words (6676635/s), 70678 word types, 3 tags
2018-11-13 21:54:08,128 : INFO : PROGRESS: at example #20000, processed 4298916 words (6320870/s), 105924 word types, 5 tags
2018-11-13 21:54:08,457 : INFO : PROGRESS: at example #30000, processed 6453383 words (6578136/s), 126722 word types, 7 tags
2018-11-13 21:54:08,840 : INFO : PROGRESS: at example #40000, processed 8869096 words (6317646/s), 153964 word types, 9 tags
2018-11-13 21:54:09,229 : INFO : PROGRESS: at example #50000, processed 11118225 words (5781067/s), 183614 word types, 9 tags
2018-11-13 21:54:09,567 : INFO : PROGRESS: at example #60000, processed 13192619 words (6140409/s), 197009 word types, 9 tags
2018-11-13 21:54:09,947 : INFO : PROGRESS: at example #70000, processed 15

0.6844444444444444


2018-11-13 21:57:01,644 : INFO : collecting all words and their counts
2018-11-13 21:57:01,645 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-11-13 21:57:01,965 : INFO : PROGRESS: at example #10000, processed 2042917 words (6402916/s), 70678 word types, 3 tags
2018-11-13 21:57:02,332 : INFO : PROGRESS: at example #20000, processed 4298916 words (6157323/s), 105924 word types, 5 tags
2018-11-13 21:57:02,669 : INFO : PROGRESS: at example #30000, processed 6453383 words (6408818/s), 126722 word types, 7 tags
2018-11-13 21:57:03,064 : INFO : PROGRESS: at example #40000, processed 8869096 words (6123389/s), 153964 word types, 9 tags
2018-11-13 21:57:03,457 : INFO : PROGRESS: at example #50000, processed 11118225 words (5737609/s), 183614 word types, 9 tags
2018-11-13 21:57:03,800 : INFO : PROGRESS: at example #60000, processed 13192619 words (6049606/s), 197009 word types, 9 tags
2018-11-13 21:57:04,189 : INFO : PROGRESS: at example #70000, processed 15

In [10]:
nlp.Doc2Vec_model.docvecs.vectors_docs.shape

(9, 300)

In [9]:
nlp.Doc2Vec_model.similar_by_word('AI')

[('지능', 0.9271109700202942),
 ('인공', 0.8999812006950378),
 ('기반', 0.8831560611724854),
 ('플랫폼', 0.8673015832901001),
 ('트랜스', 0.8506982922554016),
 ('기술', 0.8500229716300964),
 ('네트워크', 0.8473103046417236),
 ('널러', 0.8424475789070129),
 ('로보틱스', 0.8396432995796204),
 ('소프트웨어', 0.8395796418190002)]

<br>


## **set ALPHA decaying at equal speed**


In [ ]:
result_dict = {'corpus_count' : [],
               'min_count' : [],
               'vector_size' : [],
               'window' : [],
               'n_epochs' : [],
               'accuracy' : [],
              }

testing_section_ls = ['경제','기업','사회','국제','부동산','증권','정치','IT과학','문화']

# 하이퍼 파라미터 튜닝 작업 수행
train_token_for_display, train_tag_for_display = nlp.split_X_y_equally(train_df['Token'], train_df['Section'], 300)

#for size in [1000,10000, 30000]:
for size in [30000]:
    x_split, y_split = nlp.split_X_y_equally(train_df['Token'].tolist(), train_df['Section'].tolist(), size)

    for min_count in [10]:
        for vector_size in [100]:
            for window in [5]:
                
                # Doc2Vec 모델 생성
                nlp.make_Doc2Vec_model(dm = 0,
                                       min_count = min_count,
                                       sample = 1e-05,
                                       vector_size = vector_size,
                                       window = window,
                                       dm_mean = 1
                                       )
                # train at equally decaying alpha    
                for i in range(10):

                    # fix min_alpha (decaying speed)
                    nlp.Doc2Vec_model.min_alpha = nlp.Doc2Vec_model.alpha
                   
                    # build and train
                    if i == 0:
                        nlp.build_and_train_Doc2Vec_model(x_split[0], y_split[0],  n_epochs = 1)
                    
                    # train
                    else: 
                        nlp.train_Doc2Vec_model(x_split[0], y_split[0],  n_epochs = 1)
                    
                    #decay alpha and min_alpha
                    nlp.Doc2Vec_model.alpha -= 0.002
                
                nlp.Doc2Vec_model.init_sims(replace=True)
                model_name = 'Doc2Vec_dm=%s&cc=%s&vs=%s&win=%s&neg=%s&min=%s&sample=%s&epochs=%s&alpha_non-default'%(\
                                                                                                     nlp.Doc2Vec_model.dm,
                                                                                                       nlp.Doc2Vec_model.corpus_count,
                                                                                                       nlp.Doc2Vec_model.vector_size,
                                                                                                       nlp.Doc2Vec_model.window,
                                                                                                       nlp.Doc2Vec_model.negative,
                                                                                                       nlp.Doc2Vec_model.min_count,
                                                                                                       nlp.Doc2Vec_model.sample,
                                                                                                       10)
                # Doc2Vec 모델 저장
                # nlp.Doc2Vec_model.save('Doc2Vec_model/'+model_name)

            
                # PCA로 2차원 plot 생성
                '''    
                X =nlp.infer_vectors_with_Doc2Vec(train_token_for_display[0])

                from sklearn.decomposition import PCA

                pca = PCA(n_components=2)
                X_pca = pca.fit_transform(X)
                scatter_df = pd.DataFrame(X_pca,
                                          index = train_tag_for_display[0],
                                          columns = ['x','y'])

                plt.figure(figsize = (15, 15))

                for section in testing_section_ls:
                    temp_df = scatter_df[scatter_df.index == section]
                    plt.scatter(temp_df['x'].values, temp_df['y'].values, label = section, c = np.random.rand(3,))

                plt.legend(loc = 'best')
                plt.savefig('Doc2Vec_model/images/'+model_name)
                '''
                
                # 각 레이블별 1000개씩 학습, 

                # LR
                X_train = nlp.infer_vectors_with_Doc2Vec(train_token_ls_split[0])
                y_train = train_tag_ls_split[0]

                X_test = nlp.infer_vectors_with_Doc2Vec(test_token_ls_split[0])
                y_test = test_tag_ls_split[0]


                clf = LogisticRegression(solver = 'sag',
                                         multi_class = 'multinomial')


                clf.fit(X_train, y_train)
                y_pred = clf.predict(X_test)

                result_dict['corpus_count'].append(nlp.Doc2Vec_model.corpus_count)
                result_dict['min_count'].append(nlp.Doc2Vec_model.min_count)
                result_dict['vector_size'].append(nlp.Doc2Vec_model.vector_size)
                result_dict['window'].append(nlp.Doc2Vec_model.window)
                result_dict['n_epochs'].append(nlp.Doc2Vec_model.epochs)
                result_dict['accuracy'].append(accuracy_score(y_pred, y_test))

                print(accuracy_score(y_pred, y_test))

                    
#    pd.DataFrame(result_dict).to_csv('Parameter_tuning_result.csv', index = False)

In [34]:
nlp.Doc2Vec_model.docvecs.count

9

In [38]:
nlp.Doc2Vec_model.similar_by_word('경제')

[('올라설', 0.3685927987098694),
 ('컬렉터', 0.36602136492729187),
 ('현주소', 0.36005598306655884),
 ('정보기술', 0.3592676818370819),
 ('가하기', 0.3585352897644043),
 ('드는', 0.34996795654296875),
 ('달빛', 0.3458760380744934),
 ('808', 0.3432823419570923),
 ('반주', 0.340637743473053),
 ('구석기시대', 0.3382413983345032)]